In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [2]:
pref = 'profesional'

In [3]:
df1 = pd.read_csv('reto_precios.csv')
df2 = pd.read_csv('profesionales.csv', encoding='latin-1')

C:\Users\alber\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df2.head()

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,8525230,01001541310000931000000000U2,11:11 ARQUITECOS,NaN,541310,Servicios de arquitectura,0 a 5 personas,CALLE,GENERAL MIGUEL BARRAGAN,PRIVADA,GENERAL MIGUEL BARRAGAN 448,PRIVADA,GENERAL MIGUEL BARRAGAN 434,CALLE,GRECIA,436.0,NaN,NaN,NaN,NaN,NaN,COLONIA,ZONA CENTRO,NaN,NaN,NaN,20000.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,0549,21,4498041302.0,CHRISTIAN@1111ARQUITECTOS.COM,WWW.1111ARQUITECTOS.COM,Fijo,21.888813,-102.288903,2019-11
1,6905372,01001541510000971000000000U3,5TO COLOR,NaN,541510,Servicios de diseño de sistemas de cómputo y s...,0 a 5 personas,CALLE,MAR CARIBE,CALLE,MAR EGEO,AVENIDA,FUNDICION,CALLE,MAR DE LAS ANTILLAS,203.0,B,NaN,NaN,11.0,NaN,FRACCIONAMIENTO,OLIVARES SANTANA,NaN,NaN,NaN,20010.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,2009,18,NaN,5T0COLORESTUDIO@GMAIL.COM,NaN,Fijo,21.898620,-102.313587,2019-11
2,6142345,01001236113000073010000000U1,9.15 ARQUITECTOS,NaN,541310,Servicios de arquitectura,11 a 30 personas,CALLE,SIERRA DEL HUMO,AVENIDA,UNIVERSIDAD,CALLE,SIERRA DE TEPOZTLAN,CALLE,SIERRA DE LAS PALOMAS,106.0,NaN,EDIFICIO SIN NOMBRE,PISO 1,6.0,NaN,FRACCIONAMIENTO,BOSQUES DEL PRADO,EDIFICIO COMERCIAL Y DE SERVICIOS,BOSQUES DEL PARAISO,6,20130.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,2259,8,4491290879.0,GERENCIA@9.15ARQ.MX,WWW.915ARQ.MX,Fijo,21.913782,-102.312156,2010-07
3,37160,01001541110003931000000000U3,A&A DESPACHO JURIDICO,NaN,541110,Bufetes jurídicos,0 a 5 personas,CALLE,CHICHIMECO,AVENIDA,HEROE DE NACOZARI SUR,CALLE,GARABATO,CALLE,HACIENDA,117.0,203,NaN,NaN,203.0,NaN,COLONIA,LA HUERTA,NaN,NaN,NaN,20250.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,0816,43,NaN,ARIASGO@HOTMAIL.COM,NaN,Fijo,21.877087,-102.280368,2014-12
4,10599,01001541910000031001000000U8,AB MKT,AB MERCADOTECNIA SC,541910,Servicios de investigación de mercados y encue...,6 a 10 personas,BOULEVARD,LUIS DONALDO COLOSIO,CALLE,MONTE EVEREST,CALLE,SAN JUAN DE LOS LAGOS,CALLE,CERRO DE LA BUFA,111.0,NaN,NaN,NaN,203.0,203,FRACCIONAMIENTO,JARDINES DE LA CONCEPCION,NaN,NaN,NaN,20120.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,2390,19,4499128611.0,NaN,WWW.ABMERCADOTECNIA.COM,Fijo,21.924182,-102.311750,2010-07


In [5]:
df2.columns

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

In [6]:
df1 = df1[['id','lat','lon']]
# Cambiar los nombres de las variables en df1 agregando sufijo 1
df1 = df1.rename(columns={ 'lat': 'lat1', 'lon': 'lon1'})
# Cambiar los nombres de las variables en df2 agregando sufijo 2
df2 = df2.rename(columns={'id': 'id2', 'latitud': 'lat2', 'longitud': 'lon2'})


In [7]:
# Definir la tolerancia espacial (en kilómetros)
tolerancia = 0.5

# Convertir las latitudes y longitudes a radianes
df1['lat_rad1'] = df1['lat1'].apply(radians)
df1['lon_rad1'] = df1['lon1'].apply(radians)
df2['lat_rad2'] = df2['lat2'].apply(radians)
df2['lon_rad2'] = df2['lon2'].apply(radians)

In [8]:
# Calcular la distancia haversine entre las coordenadas
distances = haversine_distances(df1[['lat_rad1', 'lon_rad1']], df2[['lat_rad2', 'lon_rad2']]) * 6371000/1000  # Convertir a kilómetros

# Encontrar las coincidencias dentro de la tolerancia
matches = distances <= tolerancia

In [9]:
matching_indices = np.where(matches)

# Crear una lista para almacenar los DataFrames combinados
combined_dfs = []

# Iterar sobre los índices y columnas de coincidencias
for idx, col in zip(*matching_indices):
    # Obtener los registros coincidentes de df1 y df2
    matching_record_df1 = df1.loc[idx]
    matching_record_df2 = df2.loc[col]
    
    # Combinar los registros en un DataFrame
    combined_record = pd.concat([matching_record_df1, matching_record_df2])
    combined_dfs.append(combined_record)

# Concatenar todos los DataFrames combinados en uno solo
final_combined_df = pd.concat(combined_dfs, axis=1).T

In [10]:
final_combined_df.head(3)

,id,lat1,lon1,lat_rad1,lon_rad1,id2,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,lat2,lon2,fecha_alta,lat_rad2,lon_rad2
0,1.0,19.421317,-99.164182,0.338966,-1.730741,1027918,09015561510001883010000000U6,A&G CONSULTORES EN INOVACION TECNOLOGICA,CONSULTORES EN INNOVACIONES TECNOLOGICA SA DE CV,541610,Servicios de consultoría en administración,0 a 5 personas,CALLE,SINALOA,CALLE,3 PONIENTE (MEDELLIN),CALLE,VALLADOLID,CALLE,DURANGO,84.0,NaN,NaN,NaN,23.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,0998,4,NaN,ADMINISTRACION@C-INTEC.COM,WWW.SINTEC.COM,Fijo,19.420567,-99.167567,2010-07,0.338953,-1.730801
1,1.0,19.421317,-99.164182,0.338966,-1.730741,6732704,09015541510003504001000000U1,AGENCIA DE DISEÑO DE SOFTWARE SOFWARE & TECH,SOFTWARE AND TECH SA DE CV,541510,Servicios de diseño de sistemas de cómputo y s...,6 a 10 personas,CALLE,COLIMA,AVENIDA,OAXACA,CALLE,MEDELLIN,AVENIDA,PLAZA VILLA MADRID,325.0,NaN,NaN,NaN,NaN,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,1093,8,NaN,NaN,NaN,Fijo,19.418354,-99.166918,2016-10,0.338914,-1.730789
2,1.0,19.421317,-99.164182,0.338966,-1.730741,8326168,09015541910001121000000000U8,AGENCIA DE INVESTIGACION DE MERCADOS NET-WORK,NET-WORK MERCADOTECNIA SAS,541910,Servicios de investigación de mercados y encue...,0 a 5 personas,CALLE,URUAPAN,EJE VIAL,2 PONIENTE (MONTERREY),AVENIDA,INSURGENTES SUR,CALLE,SINALOA,7.0,NaN,EDIFICIO SIN NOMBRE,PISO 4,201.0,NaN,COLONIA,ROMA NORTE,EDIFICIO COMERCIAL Y DE SERVICIOS,URUAPAN 7,201,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,0998,13,NaN,TEYSSIER@NET-WORK.COM.MX,WWW.CAMARADEGESELL.COM.MX,Fijo,19.420912,-99.164105,2019-11,0.338959,-1.73074


In [11]:
len(final_combined_df)

163974

In [12]:
final_combined_df['numero_ext'].value_counts()

121.0     3961
42.0      3167
196.0     3098
14.0      3094
46.0      3047
          ... 
463.0        1
903.0        1
572.0        1
1521.0       1
712.0        1
Name: numero_ext, Length: 507, dtype: int64

In [13]:
final_combined_df['manzana'].value_counts()

7     11768
4     11683
8     11639
2     11394
15     8620
10     8066
13     7968
5      7260
16     7158
6      6917
1      6775
14     6063
18     6058
22     5593
3      5210
20     4992
17     4748
19     4408
12     4282
24     4000
23     3750
9      3164
11     3162
26     2924
21     2767
25     2185
27      708
46      150
44      138
37      101
29       57
45       54
41       38
39       36
28       35
33       25
35       20
40       17
34       13
32        7
30        7
48        5
43        5
42        4
Name: manzana, dtype: int64

In [14]:
final_combined_df['per_ocu'].value_counts()

0 a 5 personas        79575
11 a 30 personas      35097
6 a 10 personas       34400
31 a 50 personas       8589
251 y más personas     2678
101 a 250 personas     2214
51 a 100 personas      1421
Name: per_ocu, dtype: int64

In [15]:
personal_ocupado = {'0 a 5 personas' : 1,
'6 a 10 personas' : 2,
'11 a 30 personas' : 3,
'31 a 50 personas' : 4,
'51 a 100 personas' : 5,
'101 a 250 personas' : 6,
'251 y más personas' : 7}

final_combined_df['per_ocu'] = final_combined_df['per_ocu'].replace(personal_ocupado)
final_combined_df['per_ocu'].value_counts()

1    79575
3    35097
2    34400
4     8589
7     2678
6     2214
5     1421
Name: per_ocu, dtype: int64

In [16]:
#per_ocu
#numero_ext
#manzana

In [17]:
final_combined_df['dif_lat_rad'] =final_combined_df['lat_rad1']-final_combined_df['lat_rad2']
final_combined_df['dif_lon_rad'] =final_combined_df['lon_rad1']-final_combined_df['lon_rad2']

In [18]:
(final_combined_df['lat_rad1']-final_combined_df['lat_rad2'])

0         0.000013
1         0.000052
2         0.000007
3         0.000002
4         0.000019
            ...   
163969    0.000042
163970    0.000006
163971    0.000063
163972         0.0
163973   -0.000055
Length: 163974, dtype: object

In [19]:
final_combined_df.entidad.value_counts()

CIUDAD DE MÉXICO    163262
NUEVO LEÓN             709
BAJA CALIFORNIA          3
Name: entidad, dtype: int64

In [20]:
final_combined_df[final_combined_df.entidad.str.contains('NUEVO LEÓN')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad
9864,54.0,-0.000019,0.000028
9865,54.0,-0.000003,-0.000049
9866,54.0,0.000025,0.000009
9867,54.0,0.000014,-0.000041
9868,54.0,0.000015,-0.000035
...,...,...,...
158684,952.0,0.00004,-0.000036
158685,952.0,-0.000044,-0.000052
158686,952.0,0.000016,-0.000051
158687,952.0,0.000001,0.000074


In [21]:
final_combined_df[final_combined_df.entidad.str.contains('BAJA CALIFORNIA')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad
100976,583.0,-0.000009,0.000016
100977,583.0,0.000002,-0.000075
100978,583.0,0.000013,-0.000065


In [22]:
summary = final_combined_df[['id','per_ocu','numero_ext','manzana']]
summary.head()

,id,per_ocu,numero_ext,manzana
0,1.0,1,84.0,4
1,1.0,2,325.0,8
2,1.0,1,7.0,13
3,1.0,3,31.0,2
4,1.0,3,50.0,4


In [23]:
# Definir las columnas para la agrupación y agregación
aggregation_columns = ['per_ocu', 'numero_ext', 'manzana']  # Agrega aquí las columnas que quieras sumar

# Realizar la agrupación y las agregaciones
grouped_df = summary.groupby('id')[aggregation_columns].agg({'sum'}).reset_index()
grouped_df2 = summary.groupby('id').agg(total=('id', 'count')).reset_index()
grouped_df2.columns = ['id'] + [f'{pref}_total']



# Renombrar las columnas resultantes
grouped_df.columns = ['id'] + [f'{pref}_suma_{col}' for col in aggregation_columns]

# Mostrar el DataFrame resultante
centros = grouped_df2.merge(grouped_df)

In [24]:
centros.head()

,id,profesional_total,profesional_suma_per_ocu,profesional_suma_numero_ext,profesional_suma_manzana
0,1.0,162,374,48989.0,1554
1,2.0,168,387,46204.0,1681
2,3.0,140,252,15042.0,1604
3,4.0,105,200,13218.0,1266
4,5.0,208,337,17570.0,3349


In [25]:
centros.to_csv('centros_'+pref+'es'+'.csv')